<a href="https://colab.research.google.com/github/samarth2015/CS203_Lab_2/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This is CS203 Lab 2 assignment file

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pandera -q
!rm -rf gx
!pip install pandas
!pip install great_expectations -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 35.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.
mizani 0.

In [4]:
import numpy as np
import pandas as pd
import pandera as pa
import great_expectations as ge
import matplotlib.pyplot as plt

In [5]:
import warnings
import json

warnings.filterwarnings('ignore')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
#view citibiki trip data

citibike_data = pd.read_csv('/content/drive/MyDrive/202412-citibike-tripdata/202412-citibike-tripdata_1.csv')

citibike_data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,B44E5B10AEE58AD0,classic_bike,2024-12-14 10:58:18.153,2024-12-14 11:11:11.308,Frederick Douglass Blvd & W 145 St,7954.12,E 138 St & 5 Ave,7809.13,40.823061,-73.941928,40.814490,-73.936153,member
1,BC252DC6A6011556,electric_bike,2024-12-12 14:46:12.473,2024-12-12 16:45:37.777,Madison Ave & E 99 St,7443.01,NaN,NaN,40.789485,-73.952429,40.780000,-73.960000,member
2,6FBE55EF6FE8736D,electric_bike,2024-12-11 07:55:18.770,2024-12-11 08:02:23.460,Columbia St & Kane St,4422.05,NaN,NaN,40.687632,-74.001626,40.690000,-74.000000,member
3,908890DE7FDCF9FE,electric_bike,2024-12-09 22:51:11.668,2024-12-09 22:57:43.495,E 13 St & 2 Ave,5820.08,E 10 St & 2 Ave,5746.02,40.731539,-73.985302,40.729708,-73.986598,member
4,D5D366379A4DC0A8,classic_bike,2024-12-10 18:48:40.063,2024-12-10 19:10:32.264,11 Ave & W 41 St,6726.01,E 25 St & 1 Ave,6004.07,40.760301,-73.998842,40.738177,-73.977387,member


In [46]:
citibike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ride_id             1000000 non-null  object        
 1   rideable_type       1000000 non-null  object        
 2   started_at          1000000 non-null  datetime64[ns]
 3   ended_at            1000000 non-null  datetime64[ns]
 4   start_station_name  999375 non-null   object        
 5   start_station_id    999375 non-null   object        
 6   end_station_name    996417 non-null   object        
 7   end_station_id      995525 non-null   object        
 8   start_lat           1000000 non-null  float64       
 9   start_lng           1000000 non-null  float64       
 10  end_lat             999795 non-null   float64       
 11  end_lng             999795 non-null   float64       
 12  member_casual       1000000 non-null  object        
dtypes: datetime64

In [53]:
#check that all the ride_ids are unique
print(citibike_data.shape[0] == len(citibike_data['ride_id'].unique()))

#get the unique rideable types
print(citibike_data['rideable_type'].unique())

#get the unique member_casual
print(citibike_data['member_casual'].unique())

len(citibike_data['start_station_name'].unique())

True
['classic_bike' 'electric_bike']
['member' 'casual']


2138

In [24]:
print(type(citibike_data['started_at'][0]))

<class 'str'>


In [59]:
citibike_data['start_station_id'] = citibike_data['start_station_id'].astype(str)
citibike_data['end_station_id'] = citibike_data['end_station_id'].astype(str)

In [64]:
schema = pa.DataFrameSchema({
    'ride_id': pa.Column(pa.String, checks = [
        pa.Check(lambda s: s.str.len() == 16)
    ], nullable=False),

    'rideable_type': pa.Column(pa.String, checks = [
        pa.Check.isin(['classic_bike', 'electric_bike'])
    ], nullable=False),

    'started_at': pa.Column(pa.String, nullable=False),

    'ended_at': pa.Column(pa.String, nullable=False),

    'start_station_name': pa.Column(pa.String, nullable=False),
    'start_station_id': pa.Column(pa.String, nullable=False),

    'end_station_name': pa.Column(pa.String, nullable=False),
    'end_station_id': pa.Column(pa.String, nullable=False),

    'start_lat': pa.Column(pa.Float, checks = [
        pa.Check.ge(-90), pa.Check.le(90)
    ], nullable=False),
    'start_lng': pa.Column(pa.Float, checks = [
        pa.Check.ge(-180), pa.Check.le(180)
    ], nullable=False),

    'end_lat': pa.Column(pa.Float, checks = [
        pa.Check.ge(-90), pa.Check.le(90)
    ], nullable=False),
    'end_lng': pa.Column(pa.Float, checks = [
        pa.Check.ge(-180), pa.Check.le(180)
    ], nullable=False),

    'member_casual': pa.Column(pa.String, checks = [
        pa.Check.isin(['member', 'casual'])
    ], nullable=False),


}, drop_invalid_rows=False, strict =True)


try:
    schema.validate(citibike_data, lazy=True)
except pa.errors.SchemaErrors as e:
    print(json.dumps(e.message, indent=2))

{
  "SCHEMA": {
    "SERIES_CONTAINS_NULLS": [
      {
        "schema": null,
        "column": "start_station_name",
        "check": "not_nullable",
        "error": "non-nullable series 'start_station_name' contains null values:7183      NaN8010      NaN9244      NaN10186     NaN10791     NaN         ... 995036    NaN999616    NaN999641    NaN999668    NaN999688    NaNName: start_station_name, Length: 625, dtype: object"
      },
      {
        "schema": null,
        "column": "end_station_name",
        "check": "not_nullable",
        "error": "non-nullable series 'end_station_name' contains null values:1         NaN2         NaN204       NaN206       NaN207       NaN         ... 426414    NaN426418    NaN426421    NaN426422    NaN426424    NaNName: end_station_name, Length: 3583, dtype: object"
      },
      {
        "schema": null,
        "column": "end_lat",
        "check": "not_nullable",
        "error": "non-nullable series 'end_lat' contains null values:2143     NaN2

In [67]:
citibike_data['started_at'] = pd.to_datetime(citibike_data['started_at'])
citibike_data['ended_at'] = pd.to_datetime(citibike_data['ended_at'])

In [68]:
schema_time = pa.DataFrameSchema({
    'time_diff' : pa.Column(pa.Float, checks = [
        pa.Check.ge(0)
    ], nullable=False)
})

@pa.check_output(schema_time)
def preprocessor(dataframe):
    dataframe["time_diff"] = citibike_data["ended_at"] - citibike_data["started_at"]
    dataframe["time_diff"] = dataframe["time_diff"].dt.total_seconds()
    dataframe["time_diff"] = pd.to_numeric(dataframe["time_diff"], downcast="float")
    return dataframe

citibike_processed = preprocessor(citibike_data)

try:
    schema_time.validate(citibike_processed, lazy=True)
except pa.errors.SchemaErrors as e:
    print(json.dumps(e.message, indent=2))